In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important;} *{font-family:'Consolas'; font-size:14;} </style>"))

# 문맥을 예측해서 다음 단어 예측해보기 - RNN

sample text 출처: https://ivynet.co.kr/lorem-ipsum-korean/

In [51]:
text = """
브랜드(Brand)는 어떤 경제적인 생산자를 구별하는 지각된 이미지와 경험의 집합이며 보다 좁게는 어떤 상품이나 회사를 나타내는 상표, 표지이다.\n 숫자, 글자, 글자체, 간략화된 이미지인 로고, 색상, 구호를 포함한다.\n 브랜드는 특히 기업의 무형자산으로 소비자와 시장에서 그 기업을 나타내는 가치를 나태낸다.\n 마케팅, 광고, 홍보, 제품 디자인 등에 직접 사용되며, 문화나 경제에 있어 현대의 산업소비 사회를 나타내는 중요한 요소이기도 하다.\n 많은 연구자들은 브랜드가 일단 리더십을 얻게 되면 그것을 수십 년간 유지해왔다고 주장하고 있다.\n 경영자들이 브랜드의 리더십 유지를 중요하게 생각하는 데는 몇 가지 이유가 있다.\n 첫째, 오래된 브랜드가 고객의 관심을 받기 때문이다.\n 오래된 브랜드는 상대적으로 마케팅 비용을 적게 들여도, 프리미엄 가격을 받을 수 있다.\n 이와는 대조적으로 새 브랜드나 시장이 작은 브랜드는 시장 진출과 인지도 획득을 위해 많은 마케팅 비용을 들여야 한다.\n 둘째, 마켓리더는 규모의 경제를 이용할 수 있다.\n 따라서 좁은 틈새시장을 확보한 경쟁상대에 비해 능률적인 경영정책으로 더 높은 수익을 누릴 수 있다.\n 셋째, 이미 특정 부문에서 우위를 차지하고 있는 마켓리더는 새로운 관련 부문으로 프랜차이지를 확대할 수 있으며, 새 부문에서도 쉽게 우위를 점할 수 있다.\n 반면 리더가 되지 못한 회사들의 경우, 매년 수천 개의 새 브랜드를 출시하고 있지만 대부분이 시장에서 자취도 없이 사라진다.\n 
"""

In [72]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다"""

In [73]:
from keras_preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts([text])
encoded = t.texts_to_sequences([text])[0]

In [74]:
vocab_size = len(t.word_index) + 1
# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스의 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야 한다.

print("단어 집합의 크기 : %d" % vocab_size)

단어 집합의 크기 : 12


In [75]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [76]:
sequences = list()
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print("훈련 데이터의 개수: %d" % len(sequences))

훈련 데이터의 개수: 11


In [77]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


In [85]:
num = max(len(l) for l in sequences)

In [79]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sequences = pad_sequences(sequences, maxlen=max(len(l) for l in sequences), padding='pre')
# pad_sequences()는 모든 데이터에 대해서 0을 추가하여 길이를 맞춰줌
# maxlen의 값을 6으로 주면 모든 데이터의 길이를 sequences중 가장 길이가 긴 길이에 맞게 맞춰주며, padding의 인자로 'pre'를 주면 길이가 maxlen보다 짧은 데이터의 "앞"을 0으로 채움 

In [80]:
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]])

In [81]:
import numpy as np
sequences = np.array(sequences)
X = sequences[:, :-1] # 리스트의 마지막 열을 제외하고 저장
y = sequences[:, -1] # 리스트의 마지막 열만 저장

In [82]:
from tensorflow.keras.utils import to_categorical

# 훈련데이터를 훈련 시키기 전에 y에 대해서 원-핫 인코딩을 수행
y = to_categorical(y, num_classes=vocab_size)

In [83]:
X

array([[ 0,  0,  0,  0,  2],
       [ 0,  0,  0,  2,  3],
       [ 0,  0,  2,  3,  1],
       [ 0,  2,  3,  1,  4],
       [ 0,  0,  0,  0,  6],
       [ 0,  0,  0,  6,  1],
       [ 0,  0,  0,  0,  8],
       [ 0,  0,  0,  8,  1],
       [ 0,  0,  8,  1,  9],
       [ 0,  8,  1,  9, 10],
       [ 8,  1,  9, 10,  1]])

In [84]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.models import Sequential

model = Sequential()
# y를 제거하였으므로 이제 X의 길이는...몰라.....
# Embedding 두번째 인자는 임베딩 벡터의 출력 차원. 결과가 나오는 임베딩 벡터의 크기
model.add(Embedding(vocab_size, 10, input_length=len(X[0])))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Train on 11 samples
Epoch 1/200
11/11 - 1s - loss: 2.5149 - accuracy: 0.0000e+00
Epoch 2/200
11/11 - 0s - loss: 2.5015 - accuracy: 0.0000e+00
Epoch 3/200
11/11 - 0s - loss: 2.4884 - accuracy: 0.0909
Epoch 4/200
11/11 - 0s - loss: 2.4756 - accuracy: 0.1818
Epoch 5/200
11/11 - 0s - loss: 2.4629 - accuracy: 0.2727
Epoch 6/200
11/11 - 0s - loss: 2.4504 - accuracy: 0.1818
Epoch 7/200
11/11 - 0s - loss: 2.4376 - accuracy: 0.2727
Epoch 8/200
11/11 - 0s - loss: 2.4247 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.4113 - accuracy: 0.4545
Epoch 10/200
11/11 - 0s - loss: 2.3975 - accuracy: 0.4545
Epoch 11/200
11/11 - 0s - loss: 2.3830 - accuracy: 0.4545
Epoch 12/200
11/11 - 0s - loss: 2.3678 - accuracy: 0.4545
Epoch 13/200
11/11 - 0s - loss: 2.3518 - accuracy: 0.4545
Epoch 14/200
11/11 - 0s - loss: 2.3350 - accuracy: 0.4545
Epoch 15/200
11/11 - 0s - loss: 2.3171 - accuracy: 0.4545
Epoch 16/200
11/11 - 0s - loss: 2.2982 - accuracy: 0.4545
Epoch 17/200
11/11 - 0s - loss: 2.2782 - accuracy: 0.

Epoch 142/200
11/11 - 0s - loss: 0.3425 - accuracy: 0.9091
Epoch 143/200
11/11 - 0s - loss: 0.3359 - accuracy: 0.9091
Epoch 144/200
11/11 - 0s - loss: 0.3294 - accuracy: 0.9091
Epoch 145/200
11/11 - 0s - loss: 0.3230 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.3167 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.3105 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.3044 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2984 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2925 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2867 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.2809 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.2753 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.2698 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.2644 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.2591 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.2539 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.2488 - accuracy: 1.00

In [91]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장
        for word, index in t.word_index.items():
            if index==result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측한 단어이므로 break
            current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측한 단어를 현재 단어로 변경
            sentence = sentence + ' ' + word # 예측한 단어를 문장에 저장
        # for문이므로 이 행동을 다시 반복
        sentence = init_word + sentence
        return sentence

In [93]:
print(sentence_generation(model, t, '말이', 5))

말이
